In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F

In [2]:
hotel_1 = pd.read_csv('H1.csv',parse_dates=True,index_col='ReservationStatusDate')
hotel_2 = pd.read_csv('H2.csv',parse_dates=True,index_col='ReservationStatusDate')
hotel_1.head(10)

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus
ReservationStatusDate,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,342,2015,July,27,1,0,0,2,0,...,3,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out
2015-07-01,0,737,2015,July,27,1,0,0,2,0,...,4,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out
2015-07-02,0,7,2015,July,27,1,0,1,1,0,...,0,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out
2015-07-02,0,13,2015,July,27,1,0,1,1,0,...,0,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out
2015-07-03,0,14,2015,July,27,1,0,2,2,0,...,0,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out
2015-07-03,0,14,2015,July,27,1,0,2,2,0,...,0,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out
2015-07-03,0,0,2015,July,27,1,0,2,2,0,...,0,No Deposit,NULL,NULL,0,Transient,107.0,0,0,Check-Out
2015-07-03,0,9,2015,July,27,1,0,2,2,0,...,0,No Deposit,303,NULL,0,Transient,103.0,0,1,Check-Out
2015-05-06,1,85,2015,July,27,1,0,3,2,0,...,0,No Deposit,240,NULL,0,Transient,82.0,0,1,Canceled


In [3]:
hotel_1 = hotel_1.replace(to_replace = '       NULL', 
                 value =np.NAN) 
print(hotel_1.isna().sum())
hotel_2 = hotel_2.replace(to_replace = '       NULL', 
                 value =np.NAN) 
print(hotel_2.isna().sum())

IsCanceled                         0
LeadTime                           0
ArrivalDateYear                    0
ArrivalDateMonth                   0
ArrivalDateWeekNumber              0
ArrivalDateDayOfMonth              0
StaysInWeekendNights               0
StaysInWeekNights                  0
Adults                             0
Children                           0
Babies                             0
Meal                               0
Country                          464
MarketSegment                      0
DistributionChannel                0
IsRepeatedGuest                    0
PreviousCancellations              0
PreviousBookingsNotCanceled        0
ReservedRoomType                   0
AssignedRoomType                   0
BookingChanges                     0
DepositType                        0
Agent                           8209
Company                        36952
DaysInWaitingList                  0
CustomerType                       0
ADR                                0
R

In [4]:
# Drop Company and Agent from both hotel_1 & hotel_2 datasets
hotel_1 = hotel_1.drop(['Agent','Company'],axis=1)
hotel_2 = hotel_2.drop(['Agent','Company'],axis=1)

# Fill NA values using Most frequently occuring value in that column
hotel_1['Country'] = hotel_1['Country'].fillna(hotel_1['Country'].mode()[0])

hotel_2['Country'] = hotel_2['Country'].fillna(hotel_2['Country'].mode()[0])
hotel_2['Children'] = hotel_2['Children'].fillna(hotel_2['Children'].mode()[0])

In [5]:
# drop arrival date month
hotel_1 = hotel_1.drop(['ArrivalDateMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateMonth'],axis=1)

# drop arrival date day of month
hotel_1 = hotel_1.drop(['ArrivalDateDayOfMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateDayOfMonth'],axis=1)

# drop reservation status
hotel_1 = hotel_1.drop(['ReservationStatus'],axis=1)
hotel_2 = hotel_2.drop(['ReservationStatus'],axis=1)

In [6]:
hotel_1['AssignNewRoom'] = 0
# check if the reserved room type is different from assigned room type
hotel_1.loc[hotel_1['ReservedRoomType'] != hotel_1['AssignedRoomType'], 'AssignNewRoom'] = 1
# drop older features
hotel_1 = hotel_1.drop(['AssignedRoomType', 'ReservedRoomType'], axis=1)
hotel_1.columns

Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateWeekNumber',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'MarketSegment', 'DistributionChannel',
       'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'BookingChanges', 'DepositType',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'AssignNewRoom'],
      dtype='object')

In [7]:
# replacing 1 by True and 0 by False for treatment and outcome features
hotel_1['AssignNewRoom'] = hotel_1['AssignNewRoom'].replace({1: True, 0: False})
hotel_1['IsCanceled'] = hotel_1['IsCanceled'].replace({1: True, 0: False})

In [45]:
# a neural net with the first 3 hidden layers then separate into 2 sub-networks with 2 hidden layers computing the potential outcome in treatment and control group
class TARNet(nn.Module):
    def __init__(
        self,
        input_dim,
    ):
        super(TARNet, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(input_dim, 20),
            nn.ELU(),
            nn.Linear(20, 20),
            nn.ELU(),
            nn.Linear(20,20),
            nn.ELU()
        )
        self.regressor1 = nn.Sequential(
            nn.Linear(20, 10),
            nn.ELU(),
            nn.Linear(10, 10),
            nn.ELU(),
            nn.Linear(10, 1)
        )
        self.regressor2 = nn.Sequential(
            nn.Linear(20, 10),
            nn.ELU(),
            nn.Linear(10, 10),
            nn.ELU(),
            nn.Linear(10, 1)
        )
        
    def forward(self, x):
        x = self.linear_stack(x)
        out1 = self.regressor1(x)
        out2 = self.regressor2(x)
        concat = torch.cat((out1, out2),1)
        return concat

In [46]:
model = TARNet(1)
print(model)

TARNet(
  (linear_stack): Sequential(
    (0): Linear(in_features=1, out_features=20, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): ELU(alpha=1.0)
  )
  (regressor1): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=10, out_features=1, bias=True)
  )
  (regressor2): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [47]:
def loss(concat_true, concat_pred):
        """
        concat_true - 2 columns: outcome and treatment values
        concat_pred - 2 columns: outcome in treatment and control groups
        loss function: MSE - computed with the corresponding group (treatment or control)
        """
        y_true = concat_true[:,0] # true PO
        t_true = concat_true[:,1] # treatment value (0 or 1)

        y0_pred = concat_pred[:,0] # PO in control group
        y1_pred = concat_pred[:,1] # PO in treatment group

        # loss = t * (y1 - y_true)^2 + (1-t) * (y0 - y_true)^2
        loss = torch.sum((1-t_true) * torch.square(y0_pred - y_true) + t_true * torch.square(y1_pred - y_true))

        return loss

In [61]:
t = torch.tensor(hotel_1['AssignNewRoom'].values, dtype=torch.float32).reshape(-1,1)
y = torch.tensor(hotel_1['IsCanceled'].values, dtype=torch.float32).reshape(-1,1)
concat_true = torch.cat((y,t),1)
w = torch.tensor(hotel_1['BookingChanges'].values, dtype=torch.float32).reshape(-1,1)
w.size(dim=1).astype(torch.float32)

AttributeError: 'int' object has no attribute 'astype'

In [12]:
g = torch.Generator().manual_seed(42)
np.random.seed(42)

In [49]:
n_iter = 2000
batch_size = 32
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for i in range(n_iter+1):
    # random batch
    idx = np.random.choice(len(concat_true), batch_size)
    concat_true_batch = concat_true[idx]
    w_batch = w[idx]

    # forward pass
    concat_pred = model(w_batch)
    loss_value = loss(concat_true_batch, concat_pred)

    # backward pass
    optimizer.zero_grad()
    loss_value.backward()
    optimizer.step()

    if i % 100 == 0:
        print("Iteration " + str(i) + " loss: " + str(loss_value.item()))

Iteration 0 loss: 7.80686616897583
Iteration 100 loss: 6.806380748748779
Iteration 200 loss: 6.305943965911865
Iteration 300 loss: 4.325466632843018
Iteration 400 loss: 5.9752373695373535
Iteration 500 loss: 6.078947067260742
Iteration 600 loss: 6.51179313659668
Iteration 700 loss: 8.562907218933105
Iteration 800 loss: 7.447274208068848
Iteration 900 loss: 5.837736129760742
Iteration 1000 loss: 6.037132263183594
Iteration 1100 loss: 5.893667697906494
Iteration 1200 loss: 6.526867866516113
Iteration 1300 loss: 6.483728885650635
Iteration 1400 loss: 6.711443901062012
Iteration 1500 loss: 7.727060317993164
Iteration 1600 loss: 5.192864418029785
Iteration 1700 loss: 5.827239513397217
Iteration 1800 loss: 7.34628963470459
Iteration 1900 loss: 5.321990013122559
Iteration 2000 loss: 7.313875198364258


In [51]:
concat_pred = model(w)
y0_pred = concat_pred[:,0]
y1_pred = concat_pred[:,1]
cate_pred = y1_pred - y0_pred
ate_pred = cate_pred.mean().item()
ate_pred

-0.31180351972579956

In [15]:
# check again with tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
print(tf.__version__)

2.9.1


In [16]:
def make_tarnet(input_dim):
    '''
    The first argument is the column dimension of our data.
    It needs to be specified because the functional API creates a static computational graph
    The second argument is the strength of regularization we'll apply to the output layers
    '''
    x = Input(shape=(input_dim,), name='input')

    # REPRESENTATION
    #in TF2/Keras it is idiomatic to instantiate a layer and pass its inputs on the same line unless the layer will be reused
    #Note that we apply no regularization to the representation layers 
    phi = Dense(units=20, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=20, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=20, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=10, activation='elu',name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=10, activation='elu',name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=10, activation='elu',name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=10, activation='elu',name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None,name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None,name='y1_predictions')(y1_hidden)

    #a convenience "layer" that concatenates arrays as columns in a matrix
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions])
    #the declarations above have 
    #specified the computational graph of our network, now we instantiate it
    model = Model(inputs=x, outputs=concat_pred)

    return model

In [17]:
tarnet_model=make_tarnet(1)

In [18]:
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1

In [19]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD

In [20]:
t = np.array(hotel_1['AssignNewRoom'].values, dtype=np.float32).reshape(-1,1)
y = np.array(hotel_1['IsCanceled'].values, dtype=np.float32).reshape(-1,1)
concat_true = np.concatenate([y,t],1)
w = np.array(hotel_1['BookingChanges'].values, dtype=np.float32).reshape(-1,1)

In [22]:
val_split=0.2
batch_size=32
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
# yt = np.concatenate([data['ys'], data['t']], 1) #we'll use both y and t to compute the loss


# sgd_callbacks = [
#         TerminateOnNaN(),
#         EarlyStopping(monitor='val_loss', patience=40, min_delta=0.), 
#         #40 is Shi's recommendation for this dataset, but you should tune for your data 
#         ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
#                           min_delta=0., cooldown=0, min_lr=0),
#     ]
#optimzier hyperparameters
sgd_lr = 0.001
tarnet_model.compile(optimizer=SGD(lr=sgd_lr, nesterov=True),
                    loss=regression_loss,
                    metrics=regression_loss)

tarnet_model.fit(x=w,y=concat_true,
                # callbacks=sgd_callbacks,
                # validation_split=val_split,
                epochs=20,
                batch_size=batch_size,
                verbose=verbose)
print("DONE!")

Epoch 1/20
1252/1252 [==============================] - 14s 9ms/step - loss: 5.9320 - regression_loss: 5.9320
Epoch 2/20
1252/1252 [==============================] - 15s 12ms/step - loss: 5.9262 - regression_loss: 5.9261
Epoch 3/20
1252/1252 [==============================] - 15s 12ms/step - loss: 5.9133 - regression_loss: 5.9134
Epoch 4/20
1252/1252 [==============================] - 18s 14ms/step - loss: 5.9141 - regression_loss: 5.9141
Epoch 5/20
1252/1252 [==============================] - 12s 10ms/step - loss: 5.9037 - regression_loss: 5.9036
Epoch 6/20
1252/1252 [==============================] - 11s 8ms/step - loss: 5.9031 - regression_loss: 5.9030
Epoch 7/20
1252/1252 [==============================] - 17s 14ms/step - loss: 5.9013 - regression_loss: 5.9012
Epoch 8/20
1252/1252 [==============================] - 9s 7ms/step - loss: 5.8978 - regression_loss: 5.8978
Epoch 9/20
1252/1252 [==============================] - 9s 7ms/step - loss: 5.8911 - regression_loss: 5.8911
Epoch 1

In [24]:
concat_pred=tarnet_model.predict(w)
#dont forget to rescale the outcome before estimation!
y0_pred = concat_pred[:, 0].reshape(-1, 1)
y1_pred = concat_pred[:, 1].reshape(-1, 1)
cate_pred=y1_pred-y0_pred
ate_pred=tf.reduce_mean(cate_pred)
ate_pred

1252/1252 [==============================] - 15s 12ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=-0.27119052>